# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

In [2]:
ws = Workspace.from_config()
experiment_name = 'Capstone-cancer-experiment-hyperdrive'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'cluster-capstone'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration



In [4]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment
import os
from azureml.core.script_run_config import ScriptRunConfig
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil

if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('./train.py','./training')

#TODO: Create the different params that you will be using during training

param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# configure and submit your training run
src = ScriptRunConfig(source_directory="./training",
                      script='train.py',
                      compute_target=compute_target)

#Submit job : Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.
run = experiment.submit(src)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     policy=policy,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)

In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_310f6504-398b-4c81-883f-a920039d3f57
Web View: https://ml.azure.com/experiments/Capstone-cancer-experiment-hyperdrive/runs/HD_310f6504-398b-4c81-883f-a920039d3f57?wsid=/subscriptions/08429d2c-c480-438d-b360-41b472ce37a7/resourcegroups/ml/workspaces/udacity-projects

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-07T23:29:33.230079][API][INFO]Experiment created<END>\n""<START>[2021-04-07T23:29:33.984863][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-04-07T23:29:34.333154][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-04-07T23:29:34.9711277Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperDrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])

In [ ]:
# Code below registers the best model with the information of Metrics
model = best_run.register_model(model_name='HyperDrive_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})